Website Summarizer

In [7]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [8]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [9]:
openai = OpenAI()

In [10]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


In [11]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [12]:
ed = Website("https://fullhomegardening.com/")
print(ed.title)
print(ed.text)

Full Home Gardening - Your Guide to the World of Home Gardening
Skip to content
Full Home Gardening
Menu
Home
Gardening
Guide
About
Contact
Joseph Joseph compost bin review
by
MehrishKK
This compost bin is uniquely designed and looks fantastic to …
Read more
Leave a comment
Kohler compost bin review
by
MehrishKK
This compost bin is crafted with an innovative design and …
Read more
Leave a comment
Simplehuman compost bucket review
by
MehrishKK
This compost bin looks amazing. It is definitely constructed for a modern …
Read more
Leave a comment
THIRD ROCK COMPOST BUCKET REVIEW
by
MehrishKK
This compost bucket has a timeless design and it looks extremely …
Read more
Leave a comment
MODUODUO Composting Bin Review
by
MehrishKK
This is a double-barrel Bin that is designed for convenience …
Read more
Leave a comment
4W COMPOST CAN
by
MehrishKK
This compost can has supreme quality material and excellent workmanship. …
Read more
Leave a comment
NOW DESIGNS COMPOST BIN REVIEW
by
MehrishKK
Are yo

In [13]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
print(user_prompt_for(ed))

You are looking at a website titled Full Home Gardening - Your Guide to the World of Home Gardening
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to content
Full Home Gardening
Menu
Home
Gardening
Guide
About
Contact
Joseph Joseph compost bin review
by
MehrishKK
This compost bin is uniquely designed and looks fantastic to …
Read more
Leave a comment
Kohler compost bin review
by
MehrishKK
This compost bin is crafted with an innovative design and …
Read more
Leave a comment
Simplehuman compost bucket review
by
MehrishKK
This compost bin looks amazing. It is definitely constructed for a modern …
Read more
Leave a comment
THIRD ROCK COMPOST BUCKET REVIEW
by
MehrishKK
This compost bucket has a timeless design and it looks extremely …
Read more
Leave a comment
MODUODUO Composting Bin Review
by
MehrishKK
This is a double-barrel Bin that is designed for convenience …


In [16]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [17]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you know, just a little math magic—2 + 2 equals 4. Shocking, right?


In [18]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [19]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Full Home Gardening - Your Guide to the World of Home Gardening\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to content\nFull Home Gardening\nMenu\nHome\nGardening\nGuide\nAbout\nContact\nJoseph\xa0Joseph\xa0compost bin review\nby\nMehrishKK\nThis compost bin is uniquely designed and looks fantastic to …\nRead more\nLeave a comment\nKohler compost bin review\nby\nMehrishKK\nThis compost bin is crafted with an innovative design and …\nRead more\nLeave a comment\nSimplehuman\xa0compost bucket review\nby\nMehrishKK\nThis compost bin looks amazing. It is\xa0definitely constructed\xa0for a modern …\nR

In [20]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [21]:
summarize("https://squareyards.com")

'# Square Yards - Property in India\n\nSquare Yards is a leading real estate Proptech company in India, facilitating a seamless property-buying, renting, and management experience across a wide range of services. The platform is designed to assist users in their property journey, from transactions to financing, interiors, and property management. \n\n## Key Features:\n- **Property Listings**: A vast assortment of properties for sale and rent across various cities, including Mumbai, Delhi, Bangalore, and Pune. The site allows users to customize searches by city, budget, property type, and other preferences.\n- **Market Insights**: Provides comprehensive data intelligence on property prices, rental trends, and market conditions, helping users make informed decisions. There are interactive heatmaps that illustrate asking prices and transaction statistics across major cities.\n- **Hot Selling Projects**: Lists high-demand residential projects in major cities, featuring details on pricing a

In [22]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [24]:
display_summary("https://squareyards.com")

# Square Yards Summary

Square Yards is a leading Proptech company that operates an integrated real estate marketplace in India. The platform facilitates the entire property ownership journey, offering services that include property transactions, financing, interior design, management, and more. 

## Key Features:
- **Property Listings:** Comprehensive listings for buying and renting across major cities like Mumbai, Bangalore, Delhi, and Pune, covering various property types such as residential, commercial, and land.
- **Market Insights:** Provides detailed price insights, city-wise trends, and micro-market comparisons to aid buyers and investors in making informed decisions.
- **User Services:** Offers multiple services tailored for buyers, owners, tenants, and agents, including home loans, property legal services, and interior design.
- **Property Analysis Tools:** Features like Vastu calculators and online rent agreements enhance the user experience.
- **Technology Adoption:** Emphasizes innovation in real estate through VR/AR services and data intelligence.

## Notable Offerings:
- Hot selling projects from top developers across India.
- A mobile app for easier property searches.
- A focus on new constructions, ready-to-move homes, and investment opportunities.

Square Yards claims to be a significant player in the real estate sector, utilizing technology to simplify home buying and selling while facilitating a seamless experience.